In [1]:
# import basice apis
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import Imputer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn import svm

# import Sample Data to learn models
df_train = pd.read_csv('final_hr_analysis_train.csv', header = 0)
df_test = pd.read_csv('final_hr_analysis_test.csv', header = 0)

#ID列の削除
df_train_index = df_train["index"] 
df_test_index = df_test["index"]
df_train.drop(["index"], axis = 1, inplace = True)
df_test.drop(["index"], axis = 1, inplace = True)

#目的変数であるleftの隔離
y_train=df_train.iloc[:,[0]]
y_train=y_train.values
#y_train = df_train["left"].values
#y_train = np.array(y_train)
y_test=df_test.iloc[:,[0]]
y_test=y_test.values

df_train.drop(["left"], axis = 1, inplace = True)
df_test.drop(["left"], axis = 1, inplace = True)

# 整形(今回は０，１なので本来は不要)
#y_train = y_train.as_matrix().ravel()
#y_test = y_test.as_matrix().ravel()

#データの結合
df_all = pd.concat((df_train,df_test)).reset_index(drop=True)
print(y_train.dtype)
y_train

int64


array([[0],
       [0],
       [1],
       ...,
       [0],
       [0],
       [0]], dtype=int64)

In [2]:
#One hot encording
ohe_columns = ['sales',
               'salary']

df_all_ohe = pd.get_dummies(df_all,
                            dummy_na=True,
                            columns = ohe_columns)
print('def_all_ohe shape: (%i,%i)' %df_all_ohe.shape)
df_all_ohe.head()

def_all_ohe shape: (14999,22)


,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales_IT,sales_RandD,sales_accounting,...,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical,sales_nan,salary_high,salary_low,salary_medium,salary_nan
0,0.53,0.52,2,135,4,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,0.77,0.53,5,256,3,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
2,0.89,0.79,3,149,2,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,0.64,0.63,3,156,6,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
4,0.98,0.74,4,151,3,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [3]:
#欠損データの確認
df_all_ohe.isnull().sum()[df_all_ohe.isnull().sum() !=0].sort_values(ascending=False)

Series([], dtype: int64)

In [4]:
#欠損データの補完
imp = Imputer(missing_values ='NaN', strategy='mean', axis = 0)
imp.fit(df_all_ohe)

#学習済みImputerを適用し、X_newの欠損値を置換
df_all_ohe_columns = df_all_ohe.columns.values
df_all_ohe = pd.DataFrame(imp.transform(df_all_ohe), columns = df_all_ohe_columns)

#結果表示
df_all_ohe.shape
df_all_ohe.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales_IT,sales_RandD,sales_accounting,...,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical,sales_nan,salary_high,salary_low,salary_medium,salary_nan
0,0.53,0.52,2.0,135.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.77,0.53,5.0,256.0,3.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.89,0.79,3.0,149.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.64,0.63,3.0,156.0,6.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.98,0.74,4.0,151.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [5]:
#学習用とテスト用のデータの切り分け
n_train = df_train.shape[0]
X_train_ohe = df_all_ohe[:n_train]
X_test_ohe = df_all_ohe[n_train:]

X_train_ohe.shape
X_test_ohe.shape

#欠損データの確認
X_train_ohe.isnull().sum()[X_train_ohe.isnull().sum() !=0].sort_values(ascending=False)
#y_train.isnull().sum()[y_train.isnull().sum() !=0].sort_values(ascending=False)

Series([], dtype: int64)

In [6]:
#パイプライン
Pipe_knn = Pipeline([('scl',StandardScaler()),
                     ('est',KNeighborsClassifier())])
Pipe_logistic = Pipeline([('scl',StandardScaler()),
                          ('est',LogisticRegression(random_state=1))])
Pipe_rf = Pipeline([('scl',StandardScaler()),
                    ('est',RandomForestClassifier(random_state=1))])
Pipe_gb = Pipeline([('scl',StandardScaler()),
                    ('est',GradientBoostingClassifier(random_state=1))])
Pipe_mlp = Pipeline([('scl',StandardScaler()),
                     ('est',MLPClassifier(max_iter=2000, 
                                          random_state=1))]) #,max_iter=500
Pipe_svm = Pipeline([('scl',StandardScaler()),
                     ('est',SVC(probability = True))])

In [19]:
#グリッドサーチ用のパラメータの設定
param_gs_knn = {#'est__neighbors':[1,2,3,4,5],
                'est__weights':['uniform','distance'],
                #'est__algorithm':['auto','bal_tree','kd_tree','brute'],
                #'est__leaf_size':[10,20,30,40,50],
                #'est__p':[1,2]
               }
param_gs_logistic = {#'pca__n_components':[5,7,9],
                       'est__C':[0.1,1.0,10.0,100.0]
                       #'est__penalty':['l1','l2']
}
param_gs_rf = {'est__n_estimators':[5,10,20,50,100],
               #'est__criterion':['gini','entropy'],
               #'est__max_features':['auto','sqrt','log2'],
               'est__max_depth':[1,2,3,4,5],
               #'est__min_samples_split':[0.1,0.3,0.5,0.7,0.9],
               #'est__min_samples_leaf':[1,2,4,6,8],
               #'est__bootstrap':[True,False]
              }
param_gs_gb = {'est__loss':['deviance','exponential'],
              #'est__learing_rate':[0.001,0.01,0.1],
              'est__n_estimators':[5,10,50,100,500],
              #'est__max_depth':[1,2,3,4,5],
              #'est__criterion':['firedman_mse','mae','mse'],
              #'est__min_samples_split':[0.1,0.3,0.5,0.7,0.9],
              #'est__min_samples_leaf':[1,2,4,6,8]
              }
param_gs_mlp = {#'est__max_iter':[100,500,1000],
                #'est__hidden_layer_sizes':[(10,), (50,), (100,),
                               # (10,10,), (50,50,), (100,100,),
                               # (10, 5,), (5,5,), (30, 20, 10),
                               # (100,1000,50,), (1000,100,50,),
                               # (10,10,10), (50,50,50), (100,100,100,),
                               # ],
                 #'est__actiation':['identity', 'logistic', 'tanh', 'relu'],
                 #'est__beta_1':[0.9, 0.8, 0.7, 0.6, 0.5],
                 #'est__beta_2':[0.999, 0.9, 0.8, 0.7],
                 #'est__alpha' : [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]
                }
param_gs_svm = [{#'est__C':[1,10,100,1000],
              #'est__kernel':['rbf']},
                #{#'est__C':[1,10,100,1000],
              #'est__kernel':['linear'],
              #'est__gamma':[0.001,0.0001],
               # },
                #{'est__C':[1,10,100,1000],
                #'est__kernel':['poly'],
                #'est__gamma':[0.001,0.0001],
                #},
                #{'est__C':[1,10,100,1000],
                #'est__kernel':['sigmoid'],
                #'est__gamma':[0.001,0.0001],
                }
               ]

In [ ]:
from ipywidgets import interact,interactive,fixed,interact_manual
from IPython.display import display
import ipywidgets as widgets

def get_answer(Z):
    return Z

select_scoring = get_answer(widgets.RadioButtons(options=['f1',
                                                         'accuracy',
                                                         'precision',
                                                         'recall']))
display(select_scoring)

In [ ]:
select_scoring.value

In [8]:
#評価値はprecision,recall,accuracy等もある

#knn
print('knn 探索空間:%s' %param_gs_knn)
gs_knn = GridSearchCV(estimator=Pipe_knn,
                  param_grid=param_gs_knn,
                  scoring='f1',
                  cv=3,
                  return_train_score=False)
gs_knn = gs_knn.fit(X_train_ohe, y_train)

# 探索した結果のベストスコアとパラメータの取得
print('knn Best Score:', gs_knn.best_score_)
print('knn Best Params', gs_knn.best_params_)

knn 探索空間:{'est__weights': ['uniform', 'distance']}


C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to

knn Best Score: 0.9046441874598874
knn Best Params {'est__weights': 'distance'}


C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)


In [9]:
#rf
print('rf 探索空間:%s' %param_gs_rf)
gs_rf = GridSearchCV(estimator=Pipe_rf,
                  param_grid=param_gs_rf,
                  scoring='f1',
                  cv=3,
                  return_train_score=False)
gs_rf = gs_rf.fit(X_train_ohe, y_train)

# 探索した結果のベストスコアとパラメータの取得
print('rf Best Score:', gs_rf.best_score_)
print('rf Best Params', gs_rf.best_params_)

rf 探索空間:{'est__n_estimators': [5, 10, 20, 50, 100], 'est__max_depth': [1, 2, 3, 4, 5]}


C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:

C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n

C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n

rf Best Score: 0.8399804182718326
rf Best Params {'est__max_depth': 5, 'est__n_estimators': 10}


C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)


In [10]:
#gb
print('gb 探索空間:%s' %param_gs_gb)
gs_gb = GridSearchCV(estimator=Pipe_gb,
                  param_grid=param_gs_gb,
                  scoring='f1',
                  cv=3,
                  return_train_score=False)
gs_gb = gs_gb.fit(X_train_ohe, y_train)

# 探索した結果のベストスコアとパラメータの取得
print('gb Best Score:', gs_gb.best_score_)
print('gb Best Params', gs_gb.best_params_)

gb 探索空間:{'est__loss': ['deviance', 'exponential'], 'est__n_estimators': [5, 10, 50, 100, 500]}


C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


gb Best Score: 0.953428100060979
gb Best Params {'est__loss': 'deviance', 'est__n_estimators': 500}


In [11]:
#mlp
print('mlp 探索空間:%s' %param_gs_mlp)
gs_mlp = GridSearchCV(estimator=Pipe_mlp,
                  param_grid=param_gs_mlp,
                  scoring='f1',
                  cv=3,
                  return_train_score=False)
gs_mlp = gs_mlp.fit(X_train_ohe, y_train)

# 探索した結果のベストスコアとパラメータの取得
print('mlp Best Score:', gs_mlp.best_score_)
print('mlp Best Params', gs_mlp.best_params_)

mlp 探索空間:{}


C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was pass

mlp Best Score: 0.9157921292102302
mlp Best Params {}


In [20]:
#svm
print('svm 探索空間:%s' %param_gs_svm)
gs_svm = GridSearchCV(estimator=Pipe_svm,
                  param_grid=param_gs_svm,
                  scoring='f1',
                  cv=3,
                  return_train_score=False)
gs_svm = gs_svm.fit(X_train_ohe, y_train)

# 探索した結果のベストスコアとパラメータの取得
print('svm Best Score:', gs_svm.best_score_)
print('svm Best Params', gs_svm.best_params_)

svm 探索空間:[{}]


C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

svm Best Score: 0.8833347482009251
svm Best Params {}


In [23]:
#モデルの順位

score_model = pd.DataFrame({'MODEL NAME':['knn','logistic','rf','gb','mlp','svm'],
                           'BEST_SCORE':[gs_knn.best_score_,gs_logistic.best_score_,
                              gs_rf.best_score_,gs_gb.best_score_,
                              gs_mlp.best_score_,gs_svm.best_score_]})

score_model_rank = score_model.sort_values('BEST_SCORE',ascending=False)
print(score_model_rank)

   BEST_SCORE MODEL NAME
3    0.953428         gb
4    0.915792        mlp
0    0.904644        knn
5    0.883335        svm
2    0.839980         rf
1    0.448031   logistic


In [24]:
#学習済みモデルの保存
import pickle
from sklearn import model_selection
from sklearn.externals import joblib

filename_knn = 'Predict_knn.joblib'#sav?
joblib.dump(gs_knn.best_estimator_, filename_knn)
filename_logistic = 'Predict_logistic.joblib'#sav?
joblib.dump(gs_logistic.best_estimator_, filename_logistic)
filename_rf = 'Predict_rf.joblib'#sav?
joblib.dump(gs_rf.best_estimator_, filename_rf)
filename_gb = 'Predict_gb.joblib'#sav?
joblib.dump(gs_gb.best_estimator_, filename_gb)
filename_mlp = 'Predict_mlp.joblib'#sav?
joblib.dump(gs_mlp.best_estimator_, filename_mlp)
filename_svm = 'Predict_svm.joblib'#sav?
joblib.dump(gs_svm.best_estimator_, filename_svm)

['Predict_svm.joblib']

In [25]:
#学習済みモデルのロード
from sklearn.externals import joblib
Method_test = joblib.load(filename_gb)

In [26]:
#学習済みデータによるテストの計算
y_pred_final = Method_test.predict_proba(X_test_ohe)
submission = pd.concat((df_test_index, pd.DataFrame(y_pred_final)),axis = 1)
submission.columns = ['ID',"Prediction 0","Prediction 1"]

submission.head()

submission.to_csv("submission_gb.csv",sep=',',index=False)
submission.drop(["Prediction 0"], axis = 1, inplace = True)
submission.to_csv("aijco211.csv",sep=',',index=False)

In [ ]:
#k-hold accuracy
cv_results_knn = cross_val_score(pipe_knn,
                             X_train_ohe,
                             y_train,
                             cv=10,#kに相当する
                             scoring='accuracy')

cv_results_logistic = cross_val_score(pipe_logistic,
                             X_train_ohe,
                             y_train,
                             cv=10,#kに相当する
                             scoring='accuracy')

cv_results_rf = cross_val_score(pipe_rf,
                             X_train_ohe,
                             y_train,
                             cv=10,#kに相当する
                             scoring='accuracy') #Precision, Recall, f1, auc

cv_results_gb = cross_val_score(pipe_gb,
                             X_train_ohe,
                             y_train,
                             cv=10,#kに相当する
                             scoring='accuracy')

cv_results_mlp = cross_val_score(pipe_mlp,
                             X_train,
                             y_train,
                             cv=10,#kに相当する
                             scoring='accuracy')

print('cv_results_knn',cv_results_knn.mean(),'+-', cv_results_knn.std())
print('cv_results_logistic',cv_results_logistic.mean(),'+-', cv_results_logistic.std())
print('cv_results_rf',cv_results_rf.mean(),'+-', cv_results_rf.std())
print('cv_results_knn',cv_results_gb.mean(),'+-', cv_results_gb.std())
print('cv_results_knn',cv_results_mlp.mean(),'+-', cv_results_mlp.std())

In [ ]:
#k-hold recalling
cv_results_knn = cross_val_score(pipe_knn,
                             X_train_ohe,
                             y_train,
                             cv=10,#kに相当する
                             scoring='accuracy')

cv_results_logistic = cross_val_score(pipe_logistic,
                             X_train_ohe,
                             y_train,
                             cv=10,#kに相当する
                             scoring='accuracy')

cv_results_rf = cross_val_score(pipe_rf,
                             X_train_ohe,
                             y_train,
                             cv=10,#kに相当する
                             scoring='accuracy') #Precision, Recall, f1, auc

cv_results_gb = cross_val_score(pipe_gb,
                             X_train_ohe,
                             y_train,
                             cv=10,#kに相当する
                             scoring='accuracy')

cv_results_mlp = cross_val_score(pipe_mlp,
                             X_train,
                             y_train,
                             cv=10,#kに相当する
                             scoring='accuracy')

print('cv_results_knn',cv_results_knn.mean(),'+-', cv_results_knn.std())
print('cv_results_logistic',cv_results_logistic.mean(),'+-', cv_results_logistic.std())
print('cv_results_rf',cv_results_rf.mean(),'+-', cv_results_rf.std())
print('cv_results_knn',cv_results_gb.mean(),'+-', cv_results_gb.std())
print('cv_results_knn',cv_results_mlp.mean(),'+-', cv_results_mlp.std())

In [ ]:
estimator.get_params().keys()

In [ ]:
for param_gs_logistic in gs_logistic.get_params().keys():
    print(param_gs_logistic)

In [ ]:
clf_2=SVC(probability=True)
clf_2.fit(X_train_ohe,y_train)

In [22]:
#logostic
print('logistic 探索空間:%s' %param_gs_logistic)
gs_logistic = GridSearchCV(estimator=Pipe_logistic,
                  param_grid=param_gs_logistic,
                  scoring='f1',
                  cv=3,
                  return_train_score=False)
gs_logistic = gs_logistic.fit(X_train_ohe, y_train)

# 探索した結果のベストスコアとパラメータの取得
print('logistic Best Score:', gs_logistic.best_score_)
print('logistic Best Params', gs_logistic.best_params_)

logistic 探索空間:{'est__C': [0.1, 1.0, 10.0, 100.0]}


C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

logistic Best Score: 0.44803138797348746
logistic Best Params {'est__C': 10.0}


C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yokob\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 